In [1]:
#- 패키지 임포트
import pandas as pd
from tqdm import tqdm
import re
from konlpy.tag import Okt
okt = Okt()
from collections import Counter # 형태소별 빈도 구할 때 사용
from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_2월_전처리.txt", encoding='UTF-8')
print(df.shape, df.columns)
corpus = df['split_str'].to_list() # 문장(X문서X)으로 가져와서 실행.
# ↑ 추후에 다른 분석 시도할 때 문장 별 형태소가 필요할 때도 있음. 문장별 형태소와 이를 문서로 합친 게 있으면 편하기 때문에 문장별로 형태소 분석 실행.
print(len(corpus))

(700406, 2) Index(['split_str', 'org_idx'], dtype='object')
700406


In [3]:
''' 형태소 분석 '''
nouns = []
verbs = []
adjs = []
josas = [] # 불용어 목록에 조사 사용하기 위함.
morphs = [] # 품사 태깅 없이 형태소만 추출. 추후 TF-IDF, N-gram등 텍스트 분석은 이 데이터로 진행함.
pos_list = [] # 감성분석할 때 필요.

In [4]:
for i in tqdm(range(len(corpus))):
    # 답변에서 형태소/품사 추출
    try:
        a = okt.pos(corpus[i], norm=True, stem=True) # 단어의 정규화와 어간 추출을 실행(True).
        m = okt.morphs(corpus[i], norm=True, stem=False)
        morphs.append(m) # 형태소 추가
        pos_list.append(a) # 형태소/품사 추가
        for x, y in a:
            # 품사가 명사면 명사 리스트에 단어 추가
            if y == 'Noun':
                nouns.append(x)
            # 품사가 동사면 동사 리스트에 단어 추가
            elif y == 'Verb':
                verbs.append(x)
            # 품사가 형용사면 형용사 리스트에 단어 추가
            elif y == 'Adjective':
                adjs.append(x)
		    # 품사가 조사면 조사 리스트에 단어 추가.
            elif y == 'Josa':
                josas.append(x)
    except:
        pass

100%|██████████| 700406/700406 [11:16:17<00:00, 17.26it/s]   


In [5]:
# 명사, 동사, 형용사 리스트에서 10개 단어씩 출력해서 확인
print(nouns[0:10])
print(verbs[0:10])
print(adjs[0:10])
print(josas[0:10])

['집', '앞', '바로', '도서관', '이사', '알', '거', '구석', '진', '곳']
['오다', '돼다', '서다', '되다', '세다', '틀어놓다', '읽다', '하다', '고르다', '나오다']
['있다', '있다', '있다', '그렇다', '없다', '예쁘다', '없다', '이렇다', '좋다', '같다']
['에', '이', '게', '에', '은', '가', '랑', '의', '에는', '인데']


In [10]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(nouns))) # 이 값을 밑에 most_common 안에 넣으면 됨.
noun_cnt = Counter(nouns).most_common()

명사 = []
명사빈도 = []
for a, b in noun_cnt:
    명사.append(a)
    명사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
noun_df = pd.DataFrame({'명사':명사, '명사빈도':명사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
noun_df.to_csv("../dataset/2022/본문_형태소분석/2월/나무문화_2월_형태소분석(명사).csv", index=False, encoding='UTF-8')
noun_df

135023


,명사,명사빈도
0,것,169646
1,수,97939
2,그,89183
3,이,78736
4,사람,55512
...,...,...
135018,부현,1
135019,것치립,1
135020,꼽으셨다,1
135021,심부,1


In [11]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(verbs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
verb_cnt = Counter(verbs).most_common()

동사 = []
동사빈도 = []
for a, b in verb_cnt:
    동사.append(a)
    동사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
verb_df = pd.DataFrame({'동사':동사, '동사빈도':동사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
verb_df.to_csv("../dataset/2022/본문_형태소분석/2월/나무문화_2월_형태소분석(동사).csv", index=False, encoding='UTF-8')
verb_df

1827


,동사,동사빈도
0,하다,696338
1,되다,141029
2,보다,74209
3,않다,67996
4,되어다,60415
...,...,...
1822,버벅대다,1
1823,짜부라지다,1
1824,우그러지다,1
1825,속살거리다,1


In [12]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(adjs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
adj_cnt = Counter(adjs).most_common()

형용사 = []
형용사빈도 = []
for a, b in adj_cnt:
    형용사.append(a)
    형용사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
adj_df = pd.DataFrame({'형용사':형용사, '형용사빈도':형용사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
adj_df.to_csv("../dataset/2022/본문_형태소분석/2월/나무문화_2월_형태소분석(형용사).csv", index=False, encoding='UTF-8')
adj_df

1838


,형용사,형용사빈도
0,있다,307172
1,없다,60899
2,같다,46994
3,좋다,46464
4,이다,42111
...,...,...
1833,강마르다,1
1834,기괴망측하다,1
1835,허접스럽다,1
1836,파르스름하다,1


In [13]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(josas))) # 이 값을 밑에 most_common 안에 넣으면 됨.
josa_cnt = Counter(josas).most_common()

조사 = []
조사빈도 = []
for a, b in josa_cnt:
    조사.append(a)
    조사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
josa_df = pd.DataFrame({'조사':조사, '조사빈도':조사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
josa_df.to_csv("../dataset/2022/본문_형태소분석/2월/나무문화_2월_형태소분석(조사).csv", index=False, encoding='UTF-8')
josa_df

392


,조사,조사빈도
0,을,570630
1,의,460154
2,이,412870
3,에,397357
4,를,273115
...,...,...
387,으로서야,1
388,이라야만,1
389,으로서나,1
390,에게서처럼,1


In [14]:
df_concat = pd.concat([noun_df,verb_df,adj_df],axis=1)
df_concat.to_csv("../dataset/2022/본문_형태소분석/나무문화_2월_단어종합.txt", index=False, encoding='UTF-8')

In [15]:
df["morphs"] = morphs
df["pos_tag"] = pos_list
df

,split_str,org_idx,morphs,pos_tag
0,집 앞에 바로 도서관이 있었는데 이사 오고 1년이 다 돼서야 알게 된 거 있지 구석...,0,"[집, 앞, 에, 바로, 도서관, 이, 있었는데, 이사, 오고, 1년, 이, 다, ...","[(집, Noun), (앞, Noun), (에, Josa), (바로, Noun), ..."
1,우웅 기준 오빠 목성의 위성 에우로파에는 관심없지만 이런 사랑이라면 훌훌 떠나도 좋...,0,"[우웅, 기준, 오빠, 목성, 의, 위성, 에우로파, 에는, 관심, 없지만, 이런,...","[(우웅, Exclamation), (기준, Noun), (오빠, Noun), (목..."
2,우리반 임시 시간표인데 월요일 1교시 문학 임시라니까 언제까지 이러는 건지는 모르겠...,0,"[우리, 반, 임시, 시간표, 인데, 월요일, 1, 교시, 문학, 임시, 라니까, ...","[(우리, Noun), (반, Noun), (임시, Noun), (시간표, Noun..."
3,약간 학교를 가고 싶기도 안 가고 싶기도 함,0,"[약간, 학교, 를, 가고, 싶기도, 안, 가고, 싶기도, 함]","[(약간, Noun), (학교, Noun), (를, Josa), (가다, Verb)..."
4,물론 내 의사와는 상관없겠지만 껄껄 내 사랑 초코나무숲이 판매 종료 한다는 소식에 ...,0,"[물론, 내, 의사, 와는, 상관없겠지만, 껄껄, 내, 사랑, 초코, 나무숲, 이,...","[(물론, Adverb), (내, Noun), (의사, Noun), (와는, Jos..."
...,...,...,...,...
700401,설날 작사 작곡 윤극영 노래 가사 쓰기 재생 12 좋아요 3 00 00 01 07 ...,13352,"[설날, 작사, 작곡, 윤극영, 노래, 가사, 쓰기, 재생, 12, 좋아요, 3, ...","[(설날, Noun), (작사, Noun), (작곡, Noun), (윤극영, Nou..."
700402,아동문학가이자 동요작곡가 그리고 아동문화운동가셨던 윤극영님은 푸른 하늘 은하수 하얀...,13352,"[아동문학, 가, 이자, 동요, 작곡가, 그리고, 아동, 문화, 운동, 가셨던, 윤...","[(아동문학, Noun), (가, Josa), (이자, Noun), (동요, Nou..."
700403,일제강점기 중 우리나라 가사를 붙인 노래들을 작곡하여 어린이들에게 민족에 대한 자긍...,13352,"[일제강점기, 중, 우리나라, 가사, 를, 붙인, 노래, 들, 을, 작곡, 하여, ...","[(일제강점기, Noun), (중, Noun), (우리나라, Noun), (가사, ..."
700404,새해 복 많이 받으세요,13352,"[새해, 복, 많이, 받으세요]","[(새해, Noun), (복, Noun), (많이, Adverb), (받다, Verb)]"


In [16]:
df.to_csv("../dataset/2022/본문_형태소분석/나무문화_본문_2월_형태소 분석.txt", index=False, encoding='UTF-8')